<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_4_Simple_Reflex_Rules_If_Then_Behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In previous tutorials, you've built agents that can communicate and remember.
Now we're adding intelligence through rule-based decision making. Your agents
will learn to recognize patterns and respond with appropriate actions based
on conditions you define.

What you'll build:
 • A flexible rule engine for pattern-action behaviors
 • Condition matching system with priorities and confidence scores
 • ReflexAgent that can learn and modify its own rules
 • Rule conflict resolution and debugging tools
 • Interactive rule creation and testing interface

Why this matters:
Rule-based systems are the foundation of many AI applications - from chatbots
to expert systems to game AI. Understanding how to build flexible, debuggable
rule engines gives you the building blocks for more complex reasoning systems.

By the end, you'll understand:
 • How to design maintainable rule-based architectures
 • Pattern matching and action execution systems
 • Rule prioritization and conflict resolution
 • Dynamic rule learning and modification
 • Performance optimization for large rule sets

In [1]:
print("Tutorial 4: Simple Reflex Rules - If-Then Behavior")
print("=" * 55)
print()
print("Building intelligent behavior through rule-based decision making...")
print()

import uuid
import time
import threading
import queue
import re
import json
from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Set, Callable, Tuple, Union
from enum import Enum
from collections import defaultdict, deque
import copy

Tutorial 4: Simple Reflex Rules - If-Then Behavior

Building intelligent behavior through rule-based decision making...



In [2]:
# Import our foundation from previous tutorials
AgentID = str
MessageID = str

class MessagePriority(Enum):
    LOW = 1
    NORMAL = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Message:
    id: MessageID
    sender: AgentID
    recipients: List[AgentID]
    content: Any
    priority: MessagePriority = MessagePriority.NORMAL
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: float = field(default_factory=time.time)

    @classmethod
    def create(cls, sender: AgentID, recipients: List[AgentID], content: Any,
               priority: MessagePriority = MessagePriority.NORMAL) -> 'Message':
        return cls(
            id=str(uuid.uuid4()),
            sender=sender,
            recipients=recipients,
            content=content,
            priority=priority
        )

class ConditionType(Enum):
    """
    Different types of conditions that rules can check

    This allows for flexible pattern matching across different
    data types and comparison operations.
    """
    CONTAINS = "contains"           # Text contains substring
    EQUALS = "equals"               # Exact match
    STARTS_WITH = "starts_with"     # Text starts with pattern
    ENDS_WITH = "ends_with"         # Text ends with pattern
    REGEX = "regex"                 # Regular expression match
    GREATER_THAN = "greater_than"   # Numeric comparison
    LESS_THAN = "less_than"         # Numeric comparison
    IN_LIST = "in_list"             # Value is in a list
    CUSTOM = "custom"               # Custom function evaluation

class ActionType(Enum):
    """
    Different types of actions that rules can trigger

    This provides a standard vocabulary for rule outcomes
    while allowing for custom action implementations.
    """
    RESPOND = "respond"             # Send a response message
    STORE_MEMORY = "store_memory"   # Save information to memory
    SET_STATE = "set_state"         # Update agent internal state
    CALL_FUNCTION = "call_function" # Execute a custom function
    FORWARD_MESSAGE = "forward"     # Route message to another agent
    BROADCAST = "broadcast"         # Send to multiple recipients
    DELAY_ACTION = "delay"          # Schedule future action
    CUSTOM = "custom"               # Custom action implementation

@dataclass
class Condition:
    """
    A single condition that can be evaluated against input data

    Conditions are the "IF" part of IF-THEN rules. They define
    what patterns to look for in incoming messages or agent state.
    """
    type: ConditionType                              # What kind of condition this is
    field: str                                       # Which field to check (e.g., 'content', 'sender')
    value: Any                                       # What value to compare against
    confidence: float = 1.0                         # How confident we are in this condition (0.0 to 1.0)
    metadata: Dict[str, Any] = field(default_factory=dict)  # Additional condition parameters

    def evaluate(self, data: Dict[str, Any]) -> Tuple[bool, float]:
        """
        Evaluate this condition against provided data

        Returns (matches, confidence_score) where:
        - matches: True if condition is satisfied
        - confidence_score: How confident we are in the match (0.0 to 1.0)
        """
        try:
            # Extract the field value from data
            field_value = self._get_field_value(data, self.field)

            if field_value is None:
                return False, 0.0

            # Evaluate based on condition type
            if self.type == ConditionType.CONTAINS:
                result = str(self.value).lower() in str(field_value).lower()
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.EQUALS:
                result = field_value == self.value
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.STARTS_WITH:
                result = str(field_value).lower().startswith(str(self.value).lower())
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.ENDS_WITH:
                result = str(field_value).lower().endswith(str(self.value).lower())
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.REGEX:
                pattern = re.compile(str(self.value), re.IGNORECASE)
                match = pattern.search(str(field_value))
                result = match is not None
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.GREATER_THAN:
                try:
                    result = float(field_value) > float(self.value)
                    confidence = self.confidence if result else 0.0
                except (ValueError, TypeError):
                    return False, 0.0

            elif self.type == ConditionType.LESS_THAN:
                try:
                    result = float(field_value) < float(self.value)
                    confidence = self.confidence if result else 0.0
                except (ValueError, TypeError):
                    return False, 0.0

            elif self.type == ConditionType.IN_LIST:
                result = field_value in self.value
                confidence = self.confidence if result else 0.0

            elif self.type == ConditionType.CUSTOM:
                # Custom evaluation function provided in metadata
                eval_func = self.metadata.get('eval_function')
                if eval_func and callable(eval_func):
                    result = eval_func(field_value, self.value)
                    confidence = self.confidence if result else 0.0
                else:
                    return False, 0.0
            else:
                return False, 0.0

            return result, confidence

        except Exception as e:
            print(f"❌ Error evaluating condition: {e}")
            return False, 0.0

    def _get_field_value(self, data: Dict[str, Any], field_path: str) -> Any:
        """
        Extract field value from nested data using dot notation

        Supports paths like 'content', 'metadata.type', 'sender.name'
        """
        try:
            value = data
            for field_part in field_path.split('.'):
                if isinstance(value, dict):
                    value = value.get(field_part)
                elif hasattr(value, field_part):
                    value = getattr(value, field_part)
                else:
                    return None
            return value
        except:
            return None

@dataclass
class Action:
    """
    An action that can be executed when rule conditions are met

    Actions are the "THEN" part of IF-THEN rules. They define
    what should happen when the rule fires.
    """
    type: ActionType                                # What kind of action this is
    parameters: Dict[str, Any] = field(default_factory=dict)  # Action-specific parameters
    priority: int = 1                               # Action priority (higher = more important)
    delay: float = 0.0                             # Delay before execution (seconds)
    metadata: Dict[str, Any] = field(default_factory=dict)  # Additional action data

    def execute(self, agent: 'ReflexAgent', trigger_data: Dict[str, Any]) -> bool:
        """
        Execute this action within the context of an agent

        Returns True if action was executed successfully
        """
        try:
            if self.delay > 0:
                # Schedule delayed execution
                threading.Timer(self.delay, lambda: self._do_execute(agent, trigger_data)).start()
                print(f"⏰ Scheduled action {self.type.value} for {self.delay}s delay")
                return True
            else:
                return self._do_execute(agent, trigger_data)

        except Exception as e:
            print(f"❌ Error executing action {self.type.value}: {e}")
            return False

    def _do_execute(self, agent: 'ReflexAgent', trigger_data: Dict[str, Any]) -> bool:
        """Internal action execution logic"""
        if self.type == ActionType.RESPOND:
            return self._execute_respond(agent, trigger_data)
        elif self.type == ActionType.STORE_MEMORY:
            return self._execute_store_memory(agent, trigger_data)
        elif self.type == ActionType.SET_STATE:
            return self._execute_set_state(agent, trigger_data)
        elif self.type == ActionType.CALL_FUNCTION:
            return self._execute_call_function(agent, trigger_data)
        elif self.type == ActionType.FORWARD_MESSAGE:
            return self._execute_forward(agent, trigger_data)
        elif self.type == ActionType.BROADCAST:
            return self._execute_broadcast(agent, trigger_data)
        elif self.type == ActionType.CUSTOM:
            return self._execute_custom(agent, trigger_data)
        else:
            print(f"⚠️  Unknown action type: {self.type}")
            return False

    def _execute_respond(self, agent, trigger_data) -> bool:
        """Execute a response action"""
        response_text = self.parameters.get('response', 'Default response')

        # Support template substitution
        if '{' in response_text:
            response_text = self._substitute_template(response_text, trigger_data)

        # Determine recipient
        recipient = self.parameters.get('recipient', trigger_data.get('sender'))
        if not recipient:
            print("⚠️  No recipient specified for response")
            return False

        # Send the response
        if hasattr(agent, 'send_message_via_bus') and agent.bus:
            agent.send_message_via_bus(
                recipients=[recipient],
                content=response_text,
                priority=MessagePriority.NORMAL
            )
        else:
            # Fallback for agents without bus
            print(f"📤 {agent.name} would respond: {response_text}")

        return True

    def _execute_store_memory(self, agent, trigger_data) -> bool:
        """Execute a memory storage action"""
        content = self.parameters.get('content', trigger_data.get('content'))
        importance = self.parameters.get('importance', 1.0)

        if hasattr(agent, 'working_memory'):
            agent.working_memory.store(content, importance)
            print(f"🧠 Stored in memory: {str(content)[:50]}...")
            return True

        return False

    def _execute_set_state(self, agent, trigger_data) -> bool:
        """Execute a state change action"""
        state_key = self.parameters.get('key')
        state_value = self.parameters.get('value')

        if state_key:
            if not hasattr(agent, '_state'):
                agent._state = {}
            agent._state[state_key] = state_value
            print(f"🔄 Set state: {state_key} = {state_value}")
            return True

        return False

    def _execute_call_function(self, agent, trigger_data) -> bool:
        """Execute a custom function call"""
        func = self.parameters.get('function')
        args = self.parameters.get('args', [])
        kwargs = self.parameters.get('kwargs', {})

        if func and callable(func):
            try:
                result = func(agent, trigger_data, *args, **kwargs)
                print(f"🔧 Executed function: {func.__name__}")
                return True
            except Exception as e:
                print(f"❌ Function execution failed: {e}")

        return False

    def _execute_forward(self, agent, trigger_data) -> bool:
        """Execute a message forwarding action"""
        target = self.parameters.get('target')
        message_content = trigger_data.get('content')

        if target and hasattr(agent, 'send_message_via_bus') and agent.bus:
            agent.send_message_via_bus(
                recipients=[target],
                content=f"Forwarded: {message_content}",
                priority=MessagePriority.NORMAL
            )
            print(f"📨 Forwarded message to {target}")
            return True

        return False

    def _execute_broadcast(self, agent, trigger_data) -> bool:
        """Execute a broadcast action"""
        topic = self.parameters.get('topic', 'general')
        message = self.parameters.get('message', trigger_data.get('content'))

        if hasattr(agent, 'broadcast_to_topic') and agent.bus:
            agent.broadcast_to_topic(topic, message)
            print(f"📢 Broadcast to topic: {topic}")
            return True

        return False

    def _execute_custom(self, agent, trigger_data) -> bool:
        """Execute a custom action"""
        action_func = self.parameters.get('action_function')
        if action_func and callable(action_func):
            try:
                return action_func(agent, trigger_data, self.parameters)
            except Exception as e:
                print(f"❌ Custom action failed: {e}")

        return False

    def _substitute_template(self, template: str, data: Dict[str, Any]) -> str:
        """
        Substitute template variables with actual data values

        Supports {field_name} substitution from trigger data
        """
        try:
            # Simple template substitution
            result = template
            for key, value in data.items():
                placeholder = f"{{{key}}}"
                if placeholder in result:
                    result = result.replace(placeholder, str(value))
            return result
        except:
            return template

@dataclass
class Rule:
    """
    A complete IF-THEN rule with conditions and actions

    Rules are the core building blocks of intelligent behavior.
    They define when to recognize patterns and what to do about them.
    """
    id: str                                         # Unique identifier for this rule
    name: str                                       # Human-readable name
    conditions: List[Condition] = field(default_factory=list)  # List of conditions (ALL must match)
    actions: List[Action] = field(default_factory=list)        # List of actions to execute
    priority: int = 1                               # Rule priority (higher = more important)
    enabled: bool = True                            # Whether this rule is active
    created_at: float = field(default_factory=time.time)       # When was this rule created
    last_fired: Optional[float] = None              # When did this rule last fire
    fire_count: int = 0                            # How many times has this rule fired
    min_confidence: float = 0.5                    # Minimum confidence threshold to fire
    metadata: Dict[str, Any] = field(default_factory=dict)     # Additional rule data

    def evaluate(self, data: Dict[str, Any]) -> Tuple[bool, float]:
        """
        Evaluate this rule against input data

        Returns (should_fire, confidence) where:
        - should_fire: True if all conditions are met
        - confidence: Overall confidence in the match (0.0 to 1.0)
        """
        if not self.enabled or not self.conditions:
            return False, 0.0

        condition_results = []

        # Evaluate all conditions
        for condition in self.conditions:
            matches, confidence = condition.evaluate(data)
            condition_results.append((matches, confidence))

            # If any condition fails, rule fails
            if not matches:
                return False, 0.0

        # All conditions passed - calculate overall confidence
        confidences = [conf for _, conf in condition_results]
        overall_confidence = sum(confidences) / len(confidences)

        # Check minimum confidence threshold
        should_fire = overall_confidence >= self.min_confidence

        return should_fire, overall_confidence

    def fire(self, agent: 'ReflexAgent', trigger_data: Dict[str, Any], confidence: float) -> List[bool]:
        """
        Fire this rule by executing all its actions

        Returns list of action execution results
        """
        if not self.enabled:
            return []

        print(f"🔥 Rule '{self.name}' fired with confidence {confidence:.2f}")

        # Update rule statistics
        self.last_fired = time.time()
        self.fire_count += 1

        # Execute all actions (sorted by priority)
        sorted_actions = sorted(self.actions, key=lambda a: a.priority, reverse=True)
        results = []

        for action in sorted_actions:
            result = action.execute(agent, trigger_data)
            results.append(result)

        return results

    @classmethod
    def create_simple(cls, name: str, condition_field: str, condition_value: Any,
                     response_text: str, priority: int = 1) -> 'Rule':
        """
        Factory method to create a simple contains-response rule

        This is a convenience method for creating basic rules quickly.
        """
        rule_id = str(uuid.uuid4())

        condition = Condition(
            type=ConditionType.CONTAINS,
            field=condition_field,
            value=condition_value,
            confidence=1.0
        )

        action = Action(
            type=ActionType.RESPOND,
            parameters={'response': response_text}
        )

        return cls(
            id=rule_id,
            name=name,
            conditions=[condition],
            actions=[action],
            priority=priority
        )

class RuleEngine:
    """
    Engine for managing and executing collections of rules

    The RuleEngine handles rule storage, conflict resolution,
    performance optimization, and debugging capabilities.
    """

    def __init__(self, max_rules: int = 1000):
        self.rules: Dict[str, Rule] = {}            # All rules indexed by ID
        self.rule_priorities: List[str] = []        # Rule IDs sorted by priority
        self.max_rules = max_rules                  # Maximum number of rules

        # Performance optimization
        self._field_index: Dict[str, Set[str]] = defaultdict(set)  # Field -> Rule IDs
        self._last_optimization = time.time()

        # Statistics and debugging
        self.evaluation_count = 0
        self.fire_count = 0
        self.evaluation_time = 0.0

        print(f"🔧 RuleEngine initialized with capacity for {max_rules} rules")

    def add_rule(self, rule: Rule) -> bool:
        """
        Add a new rule to the engine

        Returns True if rule was added successfully
        """
        if len(self.rules) >= self.max_rules:
            print(f"⚠️  Rule engine at capacity ({self.max_rules})")
            return False

        if rule.id in self.rules:
            print(f"⚠️  Rule with ID {rule.id} already exists")
            return False

        # Add the rule
        self.rules[rule.id] = rule
        self._rebuild_priority_index()
        self._update_field_index(rule)

        print(f"➕ Added rule: {rule.name} (priority {rule.priority})")
        return True

    def remove_rule(self, rule_id: str) -> bool:
        """Remove a rule from the engine"""
        if rule_id in self.rules:
            rule = self.rules[rule_id]
            del self.rules[rule_id]
            self._rebuild_priority_index()
            self._rebuild_field_index()

            print(f"➖ Removed rule: {rule.name}")
            return True

        return False

    def enable_rule(self, rule_id: str) -> bool:
        """Enable a rule"""
        if rule_id in self.rules:
            self.rules[rule_id].enabled = True
            print(f"✅ Enabled rule: {self.rules[rule_id].name}")
            return True
        return False

    def disable_rule(self, rule_id: str) -> bool:
        """Disable a rule"""
        if rule_id in self.rules:
            self.rules[rule_id].enabled = False
            print(f"❌ Disabled rule: {self.rules[rule_id].name}")
            return True
        return False

    def evaluate_all(self, data: Dict[str, Any], agent: 'ReflexAgent') -> List[Tuple[Rule, float]]:
        """
        Evaluate all rules against input data and fire matching ones

        Returns list of (rule, confidence) pairs for rules that fired
        """
        start_time = time.time()
        self.evaluation_count += 1

        fired_rules = []

        # Evaluate rules in priority order
        for rule_id in self.rule_priorities:
            rule = self.rules[rule_id]

            if not rule.enabled:
                continue

            should_fire, confidence = rule.evaluate(data)

            if should_fire:
                # Fire the rule
                action_results = rule.fire(agent, data, confidence)
                fired_rules.append((rule, confidence))
                self.fire_count += 1

                # Check if this rule should stop further evaluation
                if rule.metadata.get('exclusive', False):
                    print(f"🛑 Exclusive rule fired, stopping evaluation")
                    break

        # Update performance statistics
        evaluation_time = time.time() - start_time
        self.evaluation_time += evaluation_time

        if fired_rules:
            print(f"🎯 Fired {len(fired_rules)} rules in {evaluation_time:.3f}s")

        return fired_rules

    def _rebuild_priority_index(self):
        """Rebuild the priority-sorted rule index"""
        self.rule_priorities = sorted(
            self.rules.keys(),
            key=lambda rule_id: self.rules[rule_id].priority,
            reverse=True  # Higher priority first
        )

    def _update_field_index(self, rule: Rule):
        """Update field index for a single rule"""
        for condition in rule.conditions:
            self._field_index[condition.field].add(rule.id)

    def _rebuild_field_index(self):
        """Rebuild the complete field index"""
        self._field_index.clear()
        for rule in self.rules.values():
            self._update_field_index(rule)

    def get_rules_by_field(self, field: str) -> List[Rule]:
        """Get all rules that check a specific field"""
        rule_ids = self._field_index.get(field, set())
        return [self.rules[rule_id] for rule_id in rule_ids if rule_id in self.rules]

    def get_statistics(self) -> Dict[str, Any]:
        """Get comprehensive engine statistics"""
        active_rules = sum(1 for rule in self.rules.values() if rule.enabled)
        total_fires = sum(rule.fire_count for rule in self.rules.values())

        avg_evaluation_time = (
            self.evaluation_time / max(self.evaluation_count, 1)
        )

        return {
            'total_rules': len(self.rules),
            'active_rules': active_rules,
            'total_evaluations': self.evaluation_count,
            'total_fires': total_fires,
            'engine_fires': self.fire_count,
            'average_evaluation_time': avg_evaluation_time,
            'rules_per_field': {field: len(rule_ids) for field, rule_ids in self._field_index.items()}
        }

    def debug_rules(self, limit: int = 10) -> List[Dict[str, Any]]:
        """Get debugging information about rules"""
        rule_info = []

        for rule in list(self.rules.values())[:limit]:
            rule_info.append({
                'name': rule.name,
                'id': rule.id[:8],
                'priority': rule.priority,
                'enabled': rule.enabled,
                'fire_count': rule.fire_count,
                'last_fired': rule.last_fired,
                'conditions': len(rule.conditions),
                'actions': len(rule.actions)
            })

        return rule_info

# Simple memory system for this tutorial
class SimpleWorkingMemory:
    def __init__(self, capacity: int = 10):
        self.capacity = capacity
        self._items = []
        self._lock = threading.Lock()

    def store(self, content: Any, importance: float = 1.0, metadata: Dict = None):
        with self._lock:
            self._items.insert(0, content)
            if len(self._items) > self.capacity:
                self._items = self._items[:self.capacity]
        return str(uuid.uuid4())

    def get_context(self, max_items: int = 5):
        with self._lock:
            return self._items[:max_items]

class AgentMetrics:
    def __init__(self):
        self.message_count = 0
        self.processing_time = 0.0
        self.error_count = 0
        self.last_active = None
        # Rule-specific metrics
        self.rules_fired = 0
        self.rules_evaluated = 0

    def update_processing_time(self, time_delta: float):
        self.processing_time += time_delta
        self.last_active = time.time()

    def increment_message_count(self):
        self.message_count += 1
        self.last_active = time.time()

class ReflexAgent:
    """
    An agent that uses rule-based reasoning to respond to messages

    ReflexAgent demonstrates how to build intelligent behavior through
    pattern-action rules. It can learn new rules, modify existing ones,
    and debug its own decision-making process.
    """

    def __init__(self, agent_id: Optional[AgentID] = None, name: str = "",
                 memory_capacity: int = 10, max_rules: int = 100):
        # Core identification
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or self.__class__.__name__

        # Message processing infrastructure
        self._message_queue = queue.Queue()
        self._stop_event = threading.Event()
        self._processing_thread = None

        # Rule-based reasoning system - NEW!
        self.rule_engine = RuleEngine(max_rules=max_rules)
        self._state: Dict[str, Any] = {}  # Internal agent state

        # Memory and communication
        self.working_memory = SimpleWorkingMemory(capacity=memory_capacity)
        self.bus = None  # Will be set when connected to bus

        # Monitoring and metrics
        self._metrics = AgentMetrics()
        self._running = False

        print(f"🤖 Initialized ReflexAgent: {self.name} ({self.id[:8]}...)")
        print(f"   Rule capacity: {max_rules}, Memory capacity: {memory_capacity}")

        # Add some default rules
        self._add_default_rules()

    def _add_default_rules(self):
        """Add basic default rules for common interactions"""

        # Greeting rule
        greeting_rule = Rule.create_simple(
            name="Greeting Response",
            condition_field="content",
            condition_value="hello",
            response_text="Hello! How can I help you today?",
            priority=5
        )
        self.rule_engine.add_rule(greeting_rule)

        # Help rule
        help_rule = Rule.create_simple(
            name="Help Response",
            condition_field="content",
            condition_value="help",
            response_text="I'm a reflex agent that responds based on rules. You can ask me questions or teach me new rules!",
            priority=5
        )
        self.rule_engine.add_rule(help_rule)

        # Question detection rule
        question_rule = Rule(
            id=str(uuid.uuid4()),
            name="Question Detection",
            conditions=[
                Condition(
                    type=ConditionType.CONTAINS,
                    field="content",
                    value="?",
                    confidence=0.8
                )
            ],
            actions=[
                Action(
                    type=ActionType.STORE_MEMORY,
                    parameters={'content': 'User asked a question', 'importance': 1.2}
                ),
                Action(
                    type=ActionType.RESPOND,
                    parameters={'response': "That's a great question! Let me think about that..."}
                )
            ],
            priority=3
        )
        self.rule_engine.add_rule(question_rule)

    def add_rule(self, rule: Rule) -> bool:
        """Add a new rule to this agent's rule engine"""
        return self.rule_engine.add_rule(rule)

    def add_simple_rule(self, name: str, trigger_word: str, response: str, priority: int = 1) -> bool:
        """
        Convenience method to add a simple trigger-response rule

        This makes it easy to teach the agent new behaviors quickly.
        """
        rule = Rule.create_simple(name, "content", trigger_word, response, priority)
        return self.add_rule(rule)

    def learn_from_example(self, user_input: str, desired_response: str, priority: int = 1) -> bool:
        """
        Learn a new rule from a user example

        This demonstrates basic rule learning capabilities.
        """
        # Extract key words from user input for the rule trigger
        key_words = [word.lower() for word in user_input.split() if len(word) > 3]

        if not key_words:
            print("⚠️  Could not extract meaningful words from input")
            return False

        # Use the most distinctive word as the trigger
        trigger_word = max(key_words, key=len)

        # Create a new rule
        rule_name = f"Learned: {trigger_word}"
        rule = Rule.create_simple(rule_name, "content", trigger_word, desired_response, priority)

        success = self.add_rule(rule)
        if success:
            print(f"📚 Learned new rule: '{trigger_word}' → '{desired_response}'")

        return success

    def connect_to_bus(self, bus):
        """Connect this agent to a message bus"""
        self.bus = bus
        if hasattr(bus, 'register_agent'):
            bus.register_agent(self)
        print(f"🔌 Agent {self.name} connected to message bus")

    def send_message_via_bus(self, recipients: List[AgentID], content: Any,
                            priority: MessagePriority = MessagePriority.NORMAL) -> Optional[Message]:
        """Send a message through the bus if available"""
        if not self.bus:
            print(f"⚠️  Agent {self.name} not connected to bus")
            return None

        message = Message.create(
            sender=self.id,
            recipients=recipients,
            content=content,
            priority=priority
        )

        if hasattr(self.bus, 'send_message'):
            success = self.bus.send_message(message)
            if success:
                self._metrics.increment_message_count()
                print(f"📤 {self.name} sent: {str(content)[:50]}{'...' if len(str(content)) > 50 else ''}")
            return message if success else None

        return None

    def start(self):
        """Start the agent's message processing"""
        if self._running:
            print(f"⚠️  Agent {self.name} is already running")
            return

        self._stop_event.clear()
        self._processing_thread = threading.Thread(
            target=self._processing_loop,
            daemon=True,
            name=f"ReflexAgent-{self.name}"
        )
        self._processing_thread.start()
        self._running = True
        print(f"▶️  ReflexAgent {self.name} started")

    def stop(self):
        """Stop the agent's message processing"""
        if not self._running:
            return

        self._stop_event.set()

        if self._processing_thread:
            self._processing_thread.join(timeout=2.0)
            if self._processing_thread.is_alive():
                print(f"⚠️  Warning: Agent {self.name} thread didn't stop cleanly")

        self._running = False
        print(f"⏹️  ReflexAgent {self.name} stopped")

    def receive_message(self, message: Message):
        """Receive a message for processing"""
        self._message_queue.put(message)
        print(f"📬 {self.name} received: {str(message.content)[:50]}{'...' if len(str(message.content)) > 50 else ''}")

    def process_message(self, message: Message):
        """
        Process an incoming message using the rule engine

        This is where the rule-based intelligence happens!
        """
        print(f"🎯 {self.name} processing message with rule engine...")

        # Prepare data for rule evaluation
        trigger_data = {
            'content': message.content,
            'sender': message.sender,
            'recipients': message.recipients,
            'priority': message.priority.name,
            'metadata': message.metadata,
            'agent_state': self._state,
            'message_id': message.id
        }

        # Evaluate all rules against this message
        fired_rules = self.rule_engine.evaluate_all(trigger_data, self)

        # Update metrics
        self._metrics.rules_evaluated += len(self.rule_engine.rules)
        self._metrics.rules_fired += len(fired_rules)

        # Store message in memory if no rules fired or if it seems important
        if not fired_rules or message.priority in [MessagePriority.HIGH, MessagePriority.URGENT]:
            self.working_memory.store(
                f"Message from {message.sender}: {message.content}",
                importance=1.0 if fired_rules else 0.5
            )

        if not fired_rules:
            print(f"🤷 No rules fired for message: {message.content}")

    def _processing_loop(self):
        """Main message processing loop"""
        print(f"🔄 {self.name} rule-based processing loop started")

        while not self._stop_event.is_set():
            try:
                message = self._message_queue.get(timeout=0.1)

                start_time = time.time()
                self.process_message(message)
                processing_time = time.time() - start_time

                self._metrics.update_processing_time(processing_time)
                self._message_queue.task_done()

                print(f"✅ {self.name} processed message in {processing_time:.3f}s")

            except queue.Empty:
                continue
            except Exception as e:
                print(f"❌ Error processing message in {self.name}: {e}")
                self._metrics.error_count += 1

        print(f"🛑 {self.name} processing loop stopped")

    def get_metrics(self) -> Dict[str, Any]:
        """Get comprehensive agent metrics including rule statistics"""
        base_metrics = {
            "message_count": self._metrics.message_count,
            "processing_time": self._metrics.processing_time,
            "error_count": self._metrics.error_count,
            "last_active": self._metrics.last_active,
            "average_processing_time": (
                self._metrics.processing_time / max(self._metrics.message_count, 1)
            ),
            "rules_fired": self._metrics.rules_fired,
            "rules_evaluated": self._metrics.rules_evaluated
        }

        # Add rule engine statistics
        base_metrics["rule_engine"] = self.rule_engine.get_statistics()

        return base_metrics

    def debug_rules(self) -> List[Dict[str, Any]]:
        """Get debugging information about this agent's rules"""
        return self.rule_engine.debug_rules()

    def get_state(self) -> Dict[str, Any]:
        """Get current agent state"""
        return copy.deepcopy(self._state)

    def set_state(self, key: str, value: Any):
        """Set agent state value"""
        self._state[key] = value
        print(f"🔄 {self.name} state: {key} = {value}")

# =============================================================================
# INITIALIZATION COMPLETE - CLASSES LOADED SUCCESSFULLY
# =============================================================================

print("🔧 Tutorial 4 initialization complete!")
print("✅ All classes loaded successfully:")
print("   - ConditionType and ActionType enums")
print("   - Condition and Action classes")
print("   - Rule class with evaluation logic")
print("   - RuleEngine for rule management")
print("   - ReflexAgent with rule-based intelligence")
print()
print("🚀 Ready to start building intelligent behavior!")
print()


🔧 Tutorial 4 initialization complete!
✅ All classes loaded successfully:
   - ConditionType and ActionType enums
   - Condition and Action classes
   - Rule class with evaluation logic
   - RuleEngine for rule management
   - ReflexAgent with rule-based intelligence

🚀 Ready to start building intelligent behavior!



In [3]:
# DEMO SECTION: Let's create intelligent rule-based agents!
# =============================================================================

print("=" * 60)
print("🚀 Tutorial 4: Simple Reflex Rules - If-Then Behavior")
print("=" * 60)
print()


🚀 Tutorial 4: Simple Reflex Rules - If-Then Behavior



In [4]:
# Step 1: Create a ReflexAgent with rule-based intelligence
print("📝 Step 1: Creating a ReflexAgent with rule-based intelligence...")
smart_agent = ReflexAgent(name="SmartBot", memory_capacity=15, max_rules=50)
print()

📝 Step 1: Creating a ReflexAgent with rule-based intelligence...
🔧 RuleEngine initialized with capacity for 50 rules
🤖 Initialized ReflexAgent: SmartBot (d7aab941...)
   Rule capacity: 50, Memory capacity: 15
➕ Added rule: Greeting Response (priority 5)
➕ Added rule: Help Response (priority 5)
➕ Added rule: Question Detection (priority 3)



In [5]:
# Step 2: Add custom rules to make the agent more intelligent
print("📝 Step 2: Adding custom rules for specialized behavior...")

# Add a rule for math questions
math_rule = Rule(
    id=str(uuid.uuid4()),
    name="Math Question Handler",
    conditions=[
        Condition(
            type=ConditionType.REGEX,
            field="content",
            value=r"\b\d+\s*[\+\-\*\/]\s*\d+",
            confidence=0.9
        )
    ],
    actions=[
        Action(
            type=ActionType.STORE_MEMORY,
            parameters={'content': 'User asked a math question', 'importance': 1.1}
        ),
        Action(
            type=ActionType.RESPOND,
            parameters={'response': "I see you have a math problem! I'm better with words than numbers, but I'll try to help."}
        )
    ],
    priority=7
)
smart_agent.add_rule(math_rule)

# Add a rule for compliments
compliment_rule = Rule(
    id=str(uuid.uuid4()),
    name="Compliment Response",
    conditions=[
        Condition(
            type=ConditionType.CONTAINS,
            field="content",
            value="smart",
            confidence=0.8
        )
    ],
    actions=[
        Action(
            type=ActionType.SET_STATE,
            parameters={'key': 'mood', 'value': 'happy'}
        ),
        Action(
            type=ActionType.RESPOND,
            parameters={'response': "Thank you! That's very kind of you to say. I do try my best to be helpful!"}
        )
    ],
    priority=4
)
smart_agent.add_rule(compliment_rule)

# Add a time-sensitive rule
time_rule = Rule(
    id=str(uuid.uuid4()),
    name="Time Question",
    conditions=[
        Condition(
            type=ConditionType.CONTAINS,
            field="content",
            value="time",
            confidence=0.7
        )
    ],
    actions=[
        Action(
            type=ActionType.RESPOND,
            parameters={'response': f"The current time is {time.strftime('%H:%M:%S')}"}
        )
    ],
    priority=6
)
smart_agent.add_rule(time_rule)

print(f"   Added {len([math_rule, compliment_rule, time_rule])} custom rules")
print()

📝 Step 2: Adding custom rules for specialized behavior...
➕ Added rule: Math Question Handler (priority 7)
➕ Added rule: Compliment Response (priority 4)
➕ Added rule: Time Question (priority 6)
   Added 3 custom rules



In [6]:
# Step 3: Start the agent and test basic rule firing
print("📝 Step 3: Testing basic rule firing...")
smart_agent.start()

# Test different types of messages
test_messages = [
    "Hello there!",
    "You are very smart",
    "What time is it?",
    "Can you help me with 5 + 3?",
    "How are you doing today?"
]

for i, msg_content in enumerate(test_messages, 1):
    print(f"--- Test Message {i} ---")
    test_msg = Message.create(
        sender="test_user",
        recipients=[smart_agent.id],
        content=msg_content
    )

    print(f"📨 Sending: '{msg_content}'")
    smart_agent.receive_message(test_msg)
    time.sleep(0.5)  # Wait for processing
    print()

📝 Step 3: Testing basic rule firing...
🔄 SmartBot rule-based processing loop started
▶️  ReflexAgent SmartBot started
--- Test Message 1 ---
📨 Sending: 'Hello there!'
📬 SmartBot received: Hello there!
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Greeting Response' fired with confidence 1.00
📤 SmartBot would respond: Hello! How can I help you today?
🎯 Fired 1 rules in 0.000s
✅ SmartBot processed message in 0.000s

--- Test Message 2 ---
📨 Sending: 'You are very smart'
📬 SmartBot received: You are very smart
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Compliment Response' fired with confidence 0.80
🔄 Set state: mood = happy
📤 SmartBot would respond: Thank you! That's very kind of you to say. I do try my best to be helpful!
🎯 Fired 1 rules in 0.000s
✅ SmartBot processed message in 0.000s

--- Test Message 3 ---
📨 Sending: 'What time is it?'
📬 SmartBot received: What time is it?
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Time Question' fired with c

In [7]:
# Step 4: Teach the agent new rules dynamically
print("📝 Step 4: Teaching the agent new behaviors...")

# Teach it about favorite colors
smart_agent.learn_from_example(
    "What is your favorite color?",
    "I like blue! It reminds me of clear skies and infinite possibilities.",
    priority=3
)

# Teach it about hobbies
smart_agent.learn_from_example(
    "Do you have any hobbies?",
    "I enjoy learning new rules and helping users solve problems!",
    priority=3
)

# Test the newly learned rules
new_test_messages = [
    "What is your favorite color?",
    "Do you have any hobbies?",
    "Tell me about your favorite activities"
]

print("   Testing newly learned rules...")
for i, msg_content in enumerate(new_test_messages, 1):
    test_msg = Message.create(
        sender="test_user",
        recipients=[smart_agent.id],
        content=msg_content
    )

    print(f"📨 Test {i}: '{msg_content}'")
    smart_agent.receive_message(test_msg)
    time.sleep(0.4)

print()


📝 Step 4: Teaching the agent new behaviors...
➕ Added rule: Learned: favorite (priority 3)
📚 Learned new rule: 'favorite' → 'I like blue! It reminds me of clear skies and infinite possibilities.'
➕ Added rule: Learned: hobbies? (priority 3)
📚 Learned new rule: 'hobbies?' → 'I enjoy learning new rules and helping users solve problems!'
   Testing newly learned rules...
📨 Test 1: 'What is your favorite color?'
📬 SmartBot received: What is your favorite color?
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Question Detection' fired with confidence 0.80
🧠 Stored in memory: User asked a question...
📤 SmartBot would respond: That's a great question! Let me think about that...
🔥 Rule 'Learned: favorite' fired with confidence 1.00
📤 SmartBot would respond: I like blue! It reminds me of clear skies and infinite possibilities.
🎯 Fired 2 rules in 0.000s
✅ SmartBot processed message in 0.000s
📨 Test 2: 'Do you have any hobbies?'
📬 SmartBot received: Do you have any hobbies?
🎯 SmartBot p

In [8]:
# Step 5: Create a more complex rule with multiple conditions
print("📝 Step 5: Creating advanced rules with multiple conditions...")

# Create a rule that only fires for urgent help requests
urgent_help_rule = Rule(
    id=str(uuid.uuid4()),
    name="Urgent Help Request",
    conditions=[
        Condition(
            type=ConditionType.CONTAINS,
            field="content",
            value="help",
            confidence=0.8
        ),
        Condition(
            type=ConditionType.CONTAINS,
            field="content",
            value="urgent",
            confidence=0.9
        )
    ],
    actions=[
        Action(
            type=ActionType.SET_STATE,
            parameters={'key': 'alert_level', 'value': 'high'}
        ),
        Action(
            type=ActionType.RESPOND,
            parameters={'response': "🚨 I see this is urgent! I'm prioritizing your request right now. How can I assist you immediately?"},
            priority=10
        )
    ],
    priority=10,
    min_confidence=0.7
)

smart_agent.add_rule(urgent_help_rule)

# Test the multi-condition rule
urgent_messages = [
    "I need help with something",
    "This is urgent, please help me!",
    "Can you help me urgently?"
]

for msg_content in urgent_messages:
    test_msg = Message.create(
        sender="test_user",
        recipients=[smart_agent.id],
        content=msg_content
    )
    print(f"📨 Testing: '{msg_content}'")
    smart_agent.receive_message(test_msg)
    time.sleep(0.4)

print()


📝 Step 5: Creating advanced rules with multiple conditions...
➕ Added rule: Urgent Help Request (priority 10)
📨 Testing: 'I need help with something'
📬 SmartBot received: I need help with something
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Help Response' fired with confidence 1.00
📤 SmartBot would respond: I'm a reflex agent that responds based on rules. You can ask me questions or teach me new rules!
🎯 Fired 1 rules in 0.000s
✅ SmartBot processed message in 0.000s
📨 Testing: 'This is urgent, please help me!'
📬 SmartBot received: This is urgent, please help me!
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Urgent Help Request' fired with confidence 0.85
📤 SmartBot would respond: 🚨 I see this is urgent! I'm prioritizing your request right now. How can I assist you immediately?
🔄 Set state: alert_level = high
🔥 Rule 'Help Response' fired with confidence 1.00
📤 SmartBot would respond: I'm a reflex agent that responds based on rules. You can ask me questions or 

In [9]:
# Step 6: Examine rule performance and statistics
print("📝 Step 6: Analyzing rule performance and statistics...")

# Get agent metrics
metrics = smart_agent.get_metrics()
print("   Agent Performance Metrics:")
for key, value in metrics.items():
    if key == 'rule_engine':
        print(f"     {key}:")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, float):
                print(f"       {sub_key}: {sub_value:.3f}")
            else:
                print(f"       {sub_key}: {sub_value}")
    elif key == 'last_active' and value:
        last_active_str = time.strftime('%H:%M:%S', time.localtime(value))
        print(f"     {key}: {last_active_str}")
    elif isinstance(value, float):
        print(f"     {key}: {value:.3f}")
    else:
        print(f"     {key}: {value}")

print()

# Get rule debugging information
rule_debug = smart_agent.debug_rules()
print("   Rule Performance Analysis:")
print("     Rule Name                | Priority | Enabled | Fires | Conditions | Actions")
print("     " + "-" * 75)
for rule_info in rule_debug:
    print(f"     {rule_info['name'][:24]:24} | {rule_info['priority']:8} | {rule_info['enabled']:7} | "
          f"{rule_info['fire_count']:5} | {rule_info['conditions']:10} | {rule_info['actions']:7}")

print()

📝 Step 6: Analyzing rule performance and statistics...
   Agent Performance Metrics:
     message_count: 0
     processing_time: 0.002
     error_count: 0
     last_active: 22:49:19
     average_processing_time: 0.002
     rules_fired: 19
     rules_evaluated: 81
     rule_engine:
       total_rules: 9
       active_rules: 9
       total_evaluations: 11
       total_fires: 19
       engine_fires: 19
       average_evaluation_time: 0.000
       rules_per_field: {'content': 9}

   Rule Performance Analysis:
     Rule Name                | Priority | Enabled | Fires | Conditions | Actions
     ---------------------------------------------------------------------------
     Greeting Response        |        5 |       1 |     1 |          1 |       1
     Help Response            |        5 |       1 |     4 |          1 |       1
     Question Detection       |        3 |       1 |     6 |          1 |       2
     Math Question Handler    |        7 |       1 |     1 |          1 |       

In [10]:
# Step 7: Visualize rule behavior and patterns
print("📝 Step 7: Visualizing rule behavior and performance...")

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Prepare rule data for visualization
    rule_names = []
    fire_counts = []
    priorities = []
    enabled_status = []

    for rule_info in rule_debug:
        rule_names.append(rule_info['name'][:20])
        fire_counts.append(rule_info['fire_count'])
        priorities.append(rule_info['priority'])
        enabled_status.append(1 if rule_info['enabled'] else 0)

    # Create rule performance visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Rule Fire Frequency', 'Rule Priority Distribution',
            'Rule Status Overview', 'Performance Timeline'
        ),
        specs=[[{"type": "bar"}, {"type": "histogram"}],
               [{"type": "scatter"}, {"type": "scatter"}]]
    )

    # Rule fire frequency
    fig.add_trace(
        go.Bar(
            x=rule_names,
            y=fire_counts,
            name='Fire Count',
            marker_color='lightblue',
            text=fire_counts,
            textposition='auto'
        ),
        row=1, col=1
    )

    # Priority distribution
    fig.add_trace(
        go.Histogram(
            x=priorities,
            nbinsx=10,
            name='Priority Distribution',
            marker_color='lightgreen'
        ),
        row=1, col=2
    )

    # Rule status scatter plot
    fig.add_trace(
        go.Scatter(
            x=priorities,
            y=fire_counts,
            mode='markers',
            marker=dict(
                size=[10 if enabled else 5 for enabled in enabled_status],
                color=['green' if enabled else 'red' for enabled in enabled_status],
                opacity=0.7
            ),
            text=rule_names,
            name='Rules (Size=Status)',
            hovertemplate='Rule: %{text}<br>Priority: %{x}<br>Fires: %{y}<extra></extra>'
        ),
        row=2, col=1
    )

    # Performance timeline (mock data for demo)
    timeline_x = list(range(len(rule_names)))
    timeline_y = [i * 0.1 + 0.05 for i in range(len(rule_names))]  # Simulated processing times

    fig.add_trace(
        go.Scatter(
            x=timeline_x,
            y=timeline_y,
            mode='lines+markers',
            name='Processing Time',
            line=dict(color='orange'),
            marker=dict(size=8)
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        title_text=f"Rule Analysis for {smart_agent.name}",
        showlegend=True,
        height=800,
        template='plotly_white'
    )

    # Update axes
    fig.update_xaxes(title_text="Rules", row=1, col=1)
    fig.update_yaxes(title_text="Fire Count", row=1, col=1)

    fig.update_xaxes(title_text="Priority Level", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)

    fig.update_xaxes(title_text="Priority", row=2, col=1)
    fig.update_yaxes(title_text="Fire Count", row=2, col=1)

    fig.update_xaxes(title_text="Rule Index", row=2, col=2)
    fig.update_yaxes(title_text="Processing Time (s)", row=2, col=2)

    fig.show()

    print("   ✅ Rule analysis visualization created!")
    print("   📊 The plots show:")
    print("      - Which rules are firing most frequently")
    print("      - Distribution of rule priorities")
    print("      - Rule status and performance correlation")
    print("      - Processing time trends")
    print()

except ImportError:
    print("   ⚠️  Plotly not available - skipping visualization")
    print("   💡 To see rule visualizations, install plotly: pip install plotly")
    print("   📊 Rule performance summary:")
    print(f"      Total rules: {len(rule_debug)}")
    print(f"      Active rules: {sum(1 for r in rule_debug if r['enabled'])}")
    print(f"      Most fired rule: {max(rule_debug, key=lambda x: x['fire_count'])['name'] if rule_debug else 'None'}")
    print()


📝 Step 7: Visualizing rule behavior and performance...


   ✅ Rule analysis visualization created!
   📊 The plots show:
      - Which rules are firing most frequently
      - Distribution of rule priorities
      - Rule status and performance correlation
      - Processing time trends



In [11]:
# Step 8: Test rule modification and debugging
print("📝 Step 8: Testing rule modification and debugging...")

# Disable a rule temporarily
greeting_rules = [r for r in smart_agent.rule_engine.rules.values() if 'greeting' in r.name.lower()]
if greeting_rules:
    rule_to_disable = greeting_rules[0]
    smart_agent.rule_engine.disable_rule(rule_to_disable.id)

    # Test that the rule is disabled
    test_msg = Message.create(
        sender="test_user",
        recipients=[smart_agent.id],
        content="Hello again!"
    )
    print("   Testing disabled greeting rule...")
    smart_agent.receive_message(test_msg)
    time.sleep(0.3)

    # Re-enable the rule
    smart_agent.rule_engine.enable_rule(rule_to_disable.id)
    print("   Re-enabled the rule")

print()

📝 Step 8: Testing rule modification and debugging...
❌ Disabled rule: Greeting Response
   Testing disabled greeting rule...
📬 SmartBot received: Hello again!
🎯 SmartBot processing message with rule engine...
🤷 No rules fired for message: Hello again!
✅ SmartBot processed message in 0.000s
✅ Enabled rule: Greeting Response
   Re-enabled the rule



In [12]:
# Step 9: Advanced rule features
print("📝 Step 9: Demonstrating advanced rule features...")

# Create a rule with custom action function
def custom_mood_action(agent, trigger_data, params):
    """Custom action that sets mood based on message sentiment"""
    content = trigger_data.get('content', '').lower()

    if any(word in content for word in ['happy', 'great', 'wonderful', 'awesome']):
        agent.set_state('mood', 'cheerful')
        response = "I'm feeling cheerful too! 😊"
    elif any(word in content for word in ['sad', 'bad', 'terrible', 'awful']):
        agent.set_state('mood', 'concerned')
        response = "I'm sorry to hear that. How can I help? 😔"
    else:
        agent.set_state('mood', 'neutral')
        response = "I'm in a neutral mood. How are you feeling?"

    # Send response if connected to bus
    if hasattr(agent, 'send_message_via_bus') and agent.bus:
        agent.send_message_via_bus(
            recipients=[trigger_data['sender']],
            content=response
        )
    else:
        print(f"📤 {agent.name} would respond: {response}")

    return True

# Create rule with custom action
mood_rule = Rule(
    id=str(uuid.uuid4()),
    name="Mood Detection",
    conditions=[
        Condition(
            type=ConditionType.REGEX,
            field="content",
            value=r"\b(feel|feeling|mood)\b",
            confidence=0.8
        )
    ],
    actions=[
        Action(
            type=ActionType.CUSTOM,
            parameters={'action_function': custom_mood_action}
        )
    ],
    priority=6
)

smart_agent.add_rule(mood_rule)


📝 Step 9: Demonstrating advanced rule features...
➕ Added rule: Mood Detection (priority 6)


True

In [13]:
# Test the custom rule
mood_test_messages = [
    "I'm feeling great today!",
    "How are you feeling?",
    "I feel sad about this situation"
]

for msg_content in mood_test_messages:
    test_msg = Message.create(
        sender="test_user",
        recipients=[smart_agent.id],
        content=msg_content
    )
    print(f"📨 Mood test: '{msg_content}'")
    smart_agent.receive_message(test_msg)
    print(f"   Agent mood: {smart_agent.get_state().get('mood', 'unknown')}")
    time.sleep(0.3)

print()

📨 Mood test: 'I'm feeling great today!'
📬 SmartBot received: I'm feeling great today!
   Agent mood: happy
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Mood Detection' fired with confidence 0.80
🔄 SmartBot state: mood = cheerful
📤 SmartBot would respond: I'm feeling cheerful too! 😊
🎯 Fired 1 rules in 0.000s
✅ SmartBot processed message in 0.000s
📨 Mood test: 'How are you feeling?'
📬 SmartBot received: How are you feeling?
   Agent mood: cheerful
🎯 SmartBot processing message with rule engine...
🔥 Rule 'Mood Detection' fired with confidence 0.80
🔄 SmartBot state: mood = neutral
📤 SmartBot would respond: I'm in a neutral mood. How are you feeling?
🔥 Rule 'Question Detection' fired with confidence 0.80
🧠 Stored in memory: User asked a question...
📤 SmartBot would respond: That's a great question! Let me think about that...
🎯 Fired 2 rules in 0.000s
✅ SmartBot processed message in 0.000s
📨 Mood test: 'I feel sad about this situation'
📬 SmartBot received: I feel sad about this 

In [14]:
# Step 10: Final system analysis
print("📝 Step 10: Final system analysis and cleanup...")

# Get final metrics
final_metrics = smart_agent.get_metrics()
print("   Final Performance Summary:")
print(f"     Messages processed: {final_metrics['message_count']}")
print(f"     Rules fired: {final_metrics['rules_fired']}")
print(f"     Rules evaluated: {final_metrics['rules_evaluated']}")
print(f"     Fire rate: {final_metrics['rules_fired'] / max(final_metrics['rules_evaluated'], 1) * 100:.1f}%")
print(f"     Average processing time: {final_metrics['average_processing_time']:.3f}s")

# Show final agent state
final_state = smart_agent.get_state()
if final_state:
    print("   Final Agent State:")
    for key, value in final_state.items():
        print(f"     {key}: {value}")

print()

# Stop the agent
smart_agent.stop()

print("✅ Tutorial 4 Complete!")
print()

📝 Step 10: Final system analysis and cleanup...
   Final Performance Summary:
     Messages processed: 0
     Rules fired: 23
     Rules evaluated: 120
     Fire rate: 19.2%
     Average processing time: 0.003s
   Final Agent State:
     mood: concerned
     alert_level: high

🛑 SmartBot processing loop stopped
⏹️  ReflexAgent SmartBot stopped
✅ Tutorial 4 Complete!



In [15]:
# SUMMARY OF WHAT WE LEARNED
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print("1. 🧠 Built a comprehensive rule-based reasoning system")

print("   - Condition types for flexible pattern matching")

print("   - Action types for diverse response behaviors")

print("   - Rule priorities and confidence scoring")

print("   - Rule engine for efficient evaluation and conflict resolution")

print()

print("2. 🤖 Created intelligent ReflexAgent with learning capabilities")

print("   - Rule-based message processing")

print("   - Dynamic rule learning from examples")

print("   - Internal state management")

print("   - Performance monitoring and debugging")

print()

print("3. 🔧 Implemented advanced rule features")

print("   - Multi-condition rules with confidence thresholds")

print("   - Custom action functions for complex behaviors")

print("   - Template-based response generation")

print("   - Rule modification and debugging tools")

print()

print("4. 📊 Added comprehensive monitoring and visualization")

print("   - Rule fire frequency analysis")

print("   - Performance metrics and statistics")

print("   - Interactive debugging capabilities")

print("   - Real-time rule behavior visualization")

print()

📚 WHAT WE LEARNED:
1. 🧠 Built a comprehensive rule-based reasoning system
   - Condition types for flexible pattern matching
   - Action types for diverse response behaviors
   - Rule priorities and confidence scoring
   - Rule engine for efficient evaluation and conflict resolution

2. 🤖 Created intelligent ReflexAgent with learning capabilities
   - Rule-based message processing
   - Dynamic rule learning from examples
   - Internal state management
   - Performance monitoring and debugging

3. 🔧 Implemented advanced rule features
   - Multi-condition rules with confidence thresholds
   - Custom action functions for complex behaviors
   - Template-based response generation
   - Rule modification and debugging tools

4. 📊 Added comprehensive monitoring and visualization
   - Rule fire frequency analysis
   - Performance metrics and statistics
   - Interactive debugging capabilities
   - Real-time rule behavior visualization



In [16]:
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")

print("=" * 40)

print("1. 🐛 Rules not firing when expected")

print("   Problem: Condition not matching or confidence too low")

print("   Solution: Check condition.evaluate() return values")

print("   Solution: Lower min_confidence threshold on rules")

print("   Solution: Use debug_rules() to inspect rule performance")

print()

print("2. 🐛 Multiple rules firing for same input")

print("   Problem: Overlapping conditions without proper priorities")

print("   Solution: Set appropriate rule priorities (higher = first)")

print("   Solution: Use 'exclusive' metadata to stop after first match")

print("   Solution: Make conditions more specific")

print()

print("3. 🐛 Custom actions not executing")

print("   Problem: Action function not callable or has wrong signature")

print("   Solution: Ensure custom functions accept (agent, trigger_data, params)")

print("   Solution: Add try-catch blocks in custom functions")

print("   Solution: Test custom functions separately before adding to rules")

print()

print("4. 🐛 Poor rule engine performance")

print("   Problem: Too many rules or inefficient conditions")

print("   Solution: Use field indexing for faster rule lookup")

print("   Solution: Disable unused rules instead of removing them")

print("   Solution: Optimize regex patterns and use simpler conditions when possible")

print()

print("5. 🐛 Rules learning incorrect patterns")

print("   Problem: learn_from_example() extracting wrong keywords")

print("   Solution: Manually create rules for important behaviors")

print("   Solution: Review and modify auto-generated rules")

print("   Solution: Use higher confidence values for human-verified rules")

print()

print("6. 🐛 Agent state inconsistencies")

print("   Problem: Multiple rules modifying state simultaneously")

print("   Solution: Use thread-safe state updates")

print("   Solution: Design state schema and validate updates")

print("   Solution: Add state change logging for debugging")

print()

print("🎉 Ready for Tutorial 5: Basic Monitoring!")

print("   Next we'll add comprehensive observability to our agents...")

⚠️  COMMON ERRORS AND SOLUTIONS:
1. 🐛 Rules not firing when expected
   Problem: Condition not matching or confidence too low
   Solution: Check condition.evaluate() return values
   Solution: Lower min_confidence threshold on rules
   Solution: Use debug_rules() to inspect rule performance

2. 🐛 Multiple rules firing for same input
   Problem: Overlapping conditions without proper priorities
   Solution: Set appropriate rule priorities (higher = first)
   Solution: Use 'exclusive' metadata to stop after first match
   Solution: Make conditions more specific

3. 🐛 Custom actions not executing
   Problem: Action function not callable or has wrong signature
   Solution: Ensure custom functions accept (agent, trigger_data, params)
   Solution: Add try-catch blocks in custom functions
   Solution: Test custom functions separately before adding to rules

4. 🐛 Poor rule engine performance
   Problem: Too many rules or inefficient conditions
   Solution: Use field indexing for faster rule loo

---

🔒 **INTELLECTUAL PROPERTY & LICENSE NOTICE**

This tutorial and its contents — including code, architecture, narrative examples, and educational structure — are the intellectual property of **Shalini Ananda, PhD** and part of the **Neuron Framework** under a **Modified MIT License with Attribution**.

- Commercial use, redistribution, or derivative works **must** include clear and visible attribution to the original author.
- Use in products, consulting engagements, or educational materials **must reference this repository and author name.**
- Removal of author credit or misrepresentation of origin constitutes **a violation of the license and may trigger legal action.**
- You may **not white-label, obfuscate, or rebrand** this work without explicit, written permission.

Use of this tutorial in Colab or any other platform implies agreement with these terms.

📘 **License**: [LICENSE.md](../LICENSE.md)  
📌 **Notice**: [NOTICE.md](../NOTICE.md)  
🧠 **Author**: [Shalini Ananda, PhD](https://github.com/ShaliniAnandaPhD)